![title](images/QB.png)

# Quantum Black - Data Engineering Hackathon

## Problem Statement
ACRTA road taxation data engineering


Astro City Road transport authority (ACRTA) in US have come up with an idea to use car registration renewal charges to provide indirect incentives to safe drivers. Also, providing subsidies to certain areas as per the extreme climatic conditions in terms of heavy snow or rain.

 

ACRTA has contacted us to perform a quantitative study and design a prediction model to support the aforementioned applications.

![title](images/QB1.png)

We, being a part of the data engineering team, are working continuously with the business stakeholders as well as data scientists to create features around these scenarios.

 

Problem statement that we have been provided is to “Develop inputs for a model that predicts the chances of having a vehicle accident based on driving conditions. This model will help the transport authority to understand risk patterns and act upon them.”

 

This output then would be utilized so as to come up for a risk-based taxation on different drivers and locations as per crash-prone weather conditions.

 

Use cases would be –

 

**Imposing “unsafe driving tax” on drivers to provide a positive feedback loop which may be revisited every year by looking at the past year trip data based on the driving patterns.
Lower the tax in the regions where the climatic conditions lead them to become a crash-prone site.**

 

### Data Description
**1. Drive Data (Connected car data) – Data coming from the car-mounted devices, which provides you with the car statistics every second. This information will include – Speed, acceleration, engine temperature and other car statistics.**

**2. Trip – Parameters associated with location of car such as lattitude, longitude, altitude and other similar parameters**

![title](images/trip.png)

**3. Weather – Weather condition at different latitude & longitude during different times each day.**

![title](images/weather.png)

**4. Vehicle Specifications – Different vehicle technical specifications which comes from the manufacturer of the car.**


### Submission/Output Format

Using the input data as described above, the participant would need to create features related to different hypothesis around the driving behaviours.

Solution file must be a single zip containing the 3 csvs for 3 kinds of features with the file names as given below:

**Engine Features: engine_features.csv
Drive Features: drive_features.csv
Weather Features: weather_features.csv
The csv files must have the same file names and columns in order so that checks may be done. Failure to do so would result in direct rejection as these would be tested through automated scripts.**

To understand the type of features to create, Please go through the requirements for the features carefully:

 

**1. Engine Features (file name – engine_features.csv)**
Grain - every vehicle aggregated at week start date(Monday) for the complete week in YYYY-MM-DD format.

Sorted - by Vehicle ID and week_start_Date in ascending manner

![title](images/engine.png)

Hints:

Convert timezone to PST before any calculations
All vehicles from drive data should be in the final output even if you do not have specifications (Fill with 0 if specs are not given)
Active horsepower - Engine load / 255 * Max Torque * RPM / 5252
Horsepower utilization – Active horsepower / Max Horsepower
Torque Utilization - calculated as Engine load/ 255
RPM Utilization – RPM / Maximum horsepower rpm
 

     Fields and order required in engine_features.csv –


 

**2. Drive features(file name – drive_features.csv)**
Grain – Every trip’s aggregated features at a trip id level.

Sorted - by trip_id in ascending manner
![title](images/drive.png)

Hints:

Acceleration m/s is calculated as a change in velocity over time
If a vehicle keeps on accelerating continuously over a period of time, please treat them as a single acceleration or deacceleration period.
 

Fields and order required in drive_features.csv -


 

**3. Weather features (file name – weather_features.csv)**
Here are the weather conditions for your reference and generating weather feature accordingly

![title](images/weather1.png)
![title](images/weather2.png)
Grain – Every vehicle detail should be aggregated at a week start date.

 
Sorted - by vehicle_id and week_start_date in ascending manner

![title](images/weath.png)
Hint: convert time zone to PST before any calculations


Assumptions & Hints–

Weather data is already in PST and may not need any timezone conversion. You may consider the weather data to be constant for complete hour basis. For example- if the temperature is given to be 284.51 for 2017-02-14 19:00:00, it would be the same for time 2017-02-14 19:15:45 as well.
Haversine formula must be utilized to measure the distance between any 2 consecutive points in between the trips.
Matches in between datasets must be on geohash precision point 5.
 
![title](images/hav.png)
Studies have found that - **The haversine formula** determines the great-circle distance between two points on a sphere given their longitudes and latitudes. Important in navigation, it is a special case of a more general formula in spherical trigonometry, the law of haversines, that relates the sides and angles of spherical triangles.


Fields and order required in weather_features.csv -



### Evaluation Criteria & Rules

Least deviation from the actual result would decide the accuracy of the output.
For engine and drive features, we are going to use the absolute percent deviation in between pre-calculated output versus the output provided by you. For weather features, it is going to be calculated as per mean percentage deviation from the actual output.
The final score is calculated using the deviation scores after applying weights as follows
Score = 100[W1(1- engine_deviation) + W2(1- drive_deviation) + W3(1- weather_deviation)]

Quality of code would be judged on the following parameters – functionality, reusability, modularity, documentation, testing and validation.
Should be scalable to be executed on 5 GB data as well.
Please note that scoring is going to be done using an automated script and difference in between the field names or order from the above-defined feature may result in zero scoring/error message due to the failure of the scoring script.
Participants may do multiple submissions. They would have to select on the platform which one to be treated as the final submission. If not selected, the submission with the highest score would be considered as final.
Final winners would be announced only after the submitted code reviews and the analysis of the rest of the document submissions made by the participants.
Only 5 submissions per day are allowed
 

## Submission Format
Solution checker has 2 upload links - one for solution file (described above) and 1 for code files. Final submission must be done along with code files, however, other submissions can be just made with the solution file.

 

### Code File
**Final submission must include the following relevant code files and documents. You can download a sample code file format here.**

Exploratory data analysis for the raw data – This should contain all the profiling outputs and plots generated as part of exploratory analysis. Every data source should have an individual html file with the name as that of the data source.
Data Quality issues report – Word/ PPT – High-level issues that are observed while working on the data. A single page for every data source.
Codebase (python/Java/Spark) – Code which contains a central main file for the execution of the complete pipeline and generates the final CSV outputs.
Final Model input file containing the features –
As mentioned above in the Output Format segment.
Insights if any – Word/Plot/PPT –
Do you see any information which could be useful to come to final output for the ACRTA.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input/input_data/drive"))

# Any results you write to the current directory are saved as output.

In [2]:
import pyarrow.parquet as pq
drives=os.listdir("../input/input_data/drive")
trips=os.listdir("../input/input_data/trip")
# df_trips=pd.DataF
# for t in trips:
engine=pd.read_csv("../input/sample_submission_duzbngd/engine_features.csv")
drive=pd.read_csv("../input/sample_submission_duzbngd/drive_features.csv")
weather=pd.read_csv("../input/sample_submission_duzbngd/weather_features.csv")
vehicle=pd.read_csv("../input/input_data/vehicle.csv")
vehicle.head()
drive0=pd.read_parquet('../input/input_data/drive/drive_1557398801351_trip_1557398302922_part-00000-tid-4787120893022696848-4897608f-5b93-42b5-a8ab-e90f5cadf0bf-1853-c000.snappy.parquet')
trip0=pd.read_parquet('../input/input_data/trip/trip_1557398259127_part-00000-tid-4787120893022696848-4897608f-5b93-42b5-a8ab-e90f5cadf0bf-1853-c000.snappy.parquet')

In [3]:
trip0.head()

vehicle_id                           trip_id    ...          long  velocity
0     1000504  192637d8a8a841dd85942a5ada12c50a    ...    -85.062500      0.00
1     1000504  192637d8a8a841dd85942a5ada12c50a    ...    -85.062222     59.88
2     1000504  192637d8a8a841dd85942a5ada12c50a    ...    -85.061944     76.93
3     1000504  192637d8a8a841dd85942a5ada12c50a    ...    -85.061667     75.10
4     1000504  192637d8a8a841dd85942a5ada12c50a    ...    -85.061389     48.01

[5 rows x 6 columns]

In [4]:
drive0.head()

vehicle_id                           trip_id   ...       iat      rpm
0     1000515  db3c08b01205459294bbc91e3f7dfeb6   ...      95.0  2107.44
1     1000515  db3c08b01205459294bbc91e3f7dfeb6   ...     100.0  2119.17
2     1000515  db3c08b01205459294bbc91e3f7dfeb6   ...      90.0  2112.32
3     1000515  db3c08b01205459294bbc91e3f7dfeb6   ...      90.0  2092.94
4     1000515  db3c08b01205459294bbc91e3f7dfeb6   ...      97.0  2117.87

[5 rows x 12 columns]

In [5]:
drive0.describe(include='all').T

count unique     ...               75%          max
vehicle_id           182839    NaN     ...       1.00052e+06  1.00052e+06
trip_id              182839     40     ...               NaN          NaN
datetime             182839  61719     ...               NaN          NaN
velocity             182839    NaN     ...             71.26       124.31
accel_x              182839    NaN     ...             82.52       107.15
accel_y              182839    NaN     ...             81.74       107.89
accel_z              182839    NaN     ...             80.25       107.01
engine_coolant_temp  182839    NaN     ...            147.65       198.15
eng_load             182839    NaN     ...            210.43       237.86
fuel_level           182839    NaN     ...            156.73       212.59
iat                  182839    NaN     ...           109.735       167.05
rpm                  182839    NaN     ...           2148.88      2216.92

[12 rows x 13 columns]

In [6]:
vehicle['vehicle_id']=vehicle['vehicle_id'].astype('category')

In [7]:
df_drive=[]
for d in drives:
    df_drive.append(pd.read_parquet('../input/input_data/drive/'+d)[['vehicle_id','datetime','eng_load','rpm','velocity','trip_id']])
    
dfdrive=df_drive[0]
for i in range(1,len(df_drive)):
    dfdrive=dfdrive.append(df_drive[i])
del df_drive
dfdrive.head()

vehicle_id                ...                                          trip_id
0     1000502                ...                 ef748c05ad284eedbf17115cf45ad844
1     1000502                ...                 ef748c05ad284eedbf17115cf45ad844
2     1000502                ...                 ef748c05ad284eedbf17115cf45ad844
3     1000502                ...                 ef748c05ad284eedbf17115cf45ad844
4     1000502                ...                 ef748c05ad284eedbf17115cf45ad844

[5 rows x 6 columns]

In [8]:

df_trips=[]
for t in trips:
    df_trips.append(pd.read_parquet('../input/input_data/trip/'+t)[['vehicle_id','datetime','trip_id','velocity']])
    
dftrip=df_trips[0]
for i in range(1,len(df_trips)):
    dftrip=dftrip.append(df_trips[i])
del df_trips
dftrip.head()

vehicle_id   ...    velocity
0     1000502   ...        0.00
1     1000502   ...       44.46
2     1000502   ...       58.35
3     1000502   ...       45.75
4     1000502   ...       56.45

[5 rows x 4 columns]

# Drive Features

In [9]:
dftrip['velocity']=(dftrip['velocity']*5)/18
dftrip.head()

vehicle_id    ...       velocity
0     1000502    ...       0.000000
1     1000502    ...      12.350000
2     1000502    ...      16.208333
3     1000502    ...      12.708333
4     1000502    ...      15.680556

[5 rows x 4 columns]

In [10]:
dftrip['acc']=dftrip['velocity'].diff()
dftrip['cnt_dacc']=dftrip.acc.apply(lambda x: 1 if x<0 else 0)
dftrip['cnt_acc']=dftrip.acc.apply(lambda x: 1 if x>0 else 0)

In [11]:
dftrip['cnt_dacc_10']=dftrip.acc.apply(lambda x: 1 if x<-10 else 0)
dftrip['cnt_dacc_min3_max10']=dftrip.acc.apply(lambda x: 1 if x<=-3 and x>-10 else 0)

dftrip['cnt_acc_10']=dftrip.acc.apply(lambda x: 1 if x>10 else 0)
dftrip['cnt_acc_min3_max10']=dftrip.acc.apply(lambda x: 1 if x>=3 and x<10 else 0)

In [12]:
dftrip.head(10)

vehicle_id        ...         cnt_acc_min3_max10
0     1000502        ...                          0
1     1000502        ...                          0
2     1000502        ...                          1
3     1000502        ...                          0
4     1000502        ...                          0
5     1000502        ...                          0
6     1000502        ...                          1
7     1000502        ...                          0
8     1000502        ...                          0
9     1000502        ...                          1

[10 rows x 11 columns]

In [14]:
from  functools import reduce
from itertools import groupby
def custom_count(series):
    return len([len(list(g[1])) for g in groupby(series) if g[0]==1])


In [18]:
drive.head()

trip_id             ...               ft_sum_hard_accel_3_flg_val
0  2c677d1b84a1455ca08a53770e34e024             ...                                      2021
1  2e58ce33418c4ec5b287ad35012dc9b5             ...                                      1982

[2 rows x 9 columns]

In [19]:
drive_final=dftrip.groupby('trip_id')['velocity'].sum().reset_index()[['trip_id']].merge(drive,on=['trip_id'],how='left').fillna(0)
drive_final=drive_final.drop('ft_sum_time_deaccel_val',axis=1).merge(dftrip.groupby('trip_id')['cnt_dacc'].sum().reset_index().rename(columns={'cnt_dacc':'ft_sum_time_deaccel_val'}),
                                                                     on=['trip_id'],how='left')
drive_final=drive_final.drop('ft_sum_time_accel_val',axis=1).merge(dftrip.groupby('trip_id')['cnt_acc'].sum().reset_index().rename(columns={'cnt_acc':'ft_sum_time_accel_val'}),
                                                                     on=['trip_id'],how='left')

drive_final=drive_final.drop('ft_cnt_vehicle_deaccel_val',axis=1).merge(dftrip.groupby('trip_id')['cnt_dacc'].agg(custom_count).reset_index()
                                                                        .rename(columns={'cnt_dacc':'ft_cnt_vehicle_deaccel_val'}),on=['trip_id'],how='left')

drive_final=drive_final.drop('ft_cnt_vehicle_accel_val',axis=1).merge(dftrip.groupby('trip_id')['cnt_acc'].agg(custom_count).reset_index()
                                                                        .rename(columns={'cnt_acc':'ft_cnt_vehicle_accel_val'}),
                                                                     on=['trip_id'],how='left')
drive_final.head()

trip_id            ...             ft_cnt_vehicle_accel_val
0  00922df3be5a4589ab385d0c2da2dd81            ...                                  974
1  00dc31fe55e24d14989c89de4b3b683b            ...                                 1217
2  0156d21e316d4d8b9d5bf6ccff797bf7            ...                                 1135
3  01b8a24510cd4e4684d67b96369286e0            ...                                  210
4  01c2a70c25e5428bb33811ca5eb19270            ...                                 2374

[5 rows x 9 columns]

In [20]:
drive_final=drive_final.drop('ft_sum_hard_brakes_10_flg_val',axis=1).merge(dftrip.groupby('trip_id')['cnt_dacc_10'].agg(custom_count).reset_index()
                                                                        .rename(columns={'cnt_dacc_10':'ft_sum_hard_brakes_10_flg_val'}),
                                                                     on=['trip_id'],how='left')
drive_final=drive_final.drop('ft_sum_hard_brakes_3_flg_val',axis=1).merge(dftrip.groupby('trip_id')['cnt_dacc_min3_max10'].agg(custom_count).reset_index()
                                                                        .rename(columns={'cnt_dacc_min3_max10':'ft_sum_hard_brakes_3_flg_val'}),
                                                                     on=['trip_id'],how='left')
drive_final=drive_final.drop('ft_sum_hard_accel_10_flg_val',axis=1).merge(dftrip.groupby('trip_id')['cnt_acc_10'].agg(custom_count).reset_index()
                                                                        .rename(columns={'cnt_acc_10':'ft_sum_hard_accel_10_flg_val'}),
                                                                     on=['trip_id'],how='left')
drive_final=drive_final.drop('ft_sum_hard_accel_3_flg_val',axis=1).merge(dftrip.groupby('trip_id')['cnt_acc_min3_max10'].agg(custom_count).reset_index()
                                                                        .rename(columns={'cnt_acc_min3_max10':'ft_sum_hard_accel_3_flg_val'}),
                                                                     on=['trip_id'],how='left')
drive_final.head()

trip_id             ...               ft_sum_hard_accel_3_flg_val
0  00922df3be5a4589ab385d0c2da2dd81             ...                                       520
1  00dc31fe55e24d14989c89de4b3b683b             ...                                       816
2  0156d21e316d4d8b9d5bf6ccff797bf7             ...                                       230
3  01b8a24510cd4e4684d67b96369286e0             ...                                       146
4  01c2a70c25e5428bb33811ca5eb19270             ...                                       683

[5 rows x 9 columns]

In [21]:
drive_final.to_csv('drive_features.csv',index=False)

# Engine Features

In [22]:
from datetime import datetime
from pytz import timezone
import pytz
def convert_utc_to_pst(d):
    d = pytz.UTC.localize(d)
    date = d.astimezone(timezone('America/Los_Angeles'))
    return date.strftime("%Y-%m-%d %H:%M:%S")

In [23]:
dfdrive.datetime.values

array(['2017-01-27T12:00:00.000000000', '2017-01-27T12:00:01.000000000',
       '2017-01-27T12:00:02.000000000', ...,
       '2017-01-02T21:36:08.000000000', '2017-01-02T21:36:09.000000000',
       '2017-01-02T21:36:10.000000000'], dtype='datetime64[ns]')

In [24]:
from tqdm import tqdm_notebook,tqdm

tqdm.pandas()
dfdrive.datetime=dfdrive.datetime.progress_apply(convert_utc_to_pst)

100%|██████████| 9217528/9217528 [06:05<00:00, 25249.37it/s]


In [25]:
# pd.to_datetime(dfdrive.datetime).dt.date

In [26]:
dfdrive['week_start_date']=pd.to_datetime(dfdrive.datetime).dt.date
dfdrive['week_start_date'] = pd.to_datetime(dfdrive['week_start_date'])


In [27]:
dfdrive['vehicle_id']=dfdrive['vehicle_id'].astype('category')

In [28]:
dfdrive.head()

vehicle_id       ...       week_start_date
0    1000502       ...            2017-01-27
1    1000502       ...            2017-01-27
2    1000502       ...            2017-01-27
3    1000502       ...            2017-01-27
4    1000502       ...            2017-01-27

[5 rows x 7 columns]

In [29]:
dfdrive['torque']=dfdrive['eng_load']/255
dfdrive.head()

vehicle_id             datetime    ...     week_start_date    torque
0    1000502  2017-01-27 04:00:00    ...          2017-01-27  0.782392
1    1000502  2017-01-27 04:00:01    ...          2017-01-27  0.750118
2    1000502  2017-01-27 04:00:02    ...          2017-01-27  0.772941
3    1000502  2017-01-27 04:00:03    ...          2017-01-27  0.791059
4    1000502  2017-01-27 04:00:04    ...          2017-01-27  0.741725

[5 rows x 8 columns]

In [30]:
dfdrive.shape

(9217528, 8)

In [31]:
dfdrive=dfdrive.merge(vehicle[['vehicle_id','max_torque','max_horsepower','max_horsepower_rpm']], on=['vehicle_id'],how='left')
dfdrive.tail()

vehicle_id        ...         max_horsepower_rpm
9217523     1000505        ...                        NaN
9217524     1000505        ...                        NaN
9217525     1000505        ...                        NaN
9217526     1000505        ...                        NaN
9217527     1000505        ...                        NaN

[5 rows x 11 columns]

In [32]:
def count_torque(v):
    if v>=0.6 and v<0.7:
        return 1
    elif v>=0.7 and v<0.8:
        return 2
    elif v>=0.8 and v<0.9:
        return 3
    elif v>=0.9:
        return 4
dfdrive['torque_bin']=dfdrive['torque'].apply(count_torque)
dfdrive.head()

vehicle_id             datetime     ...      max_horsepower_rpm  torque_bin
0     1000502  2017-01-27 04:00:00     ...                  4000.0           2
1     1000502  2017-01-27 04:00:01     ...                  4000.0           2
2     1000502  2017-01-27 04:00:02     ...                  4000.0           2
3     1000502  2017-01-27 04:00:03     ...                  4000.0           2
4     1000502  2017-01-27 04:00:04     ...                  4000.0           2

[5 rows x 12 columns]

In [33]:
dfdrive['act_hp']=dfdrive['eng_load']/255*dfdrive['max_torque']*dfdrive['rpm']/5252
dfdrive['act_hp_util']=dfdrive['act_hp']/dfdrive['max_horsepower']
dfdrive.tail()

vehicle_id             datetime     ...       act_hp  act_hp_util
9217523     1000505  2017-01-02 13:36:06     ...          NaN          NaN
9217524     1000505  2017-01-02 13:36:07     ...          NaN          NaN
9217525     1000505  2017-01-02 13:36:08     ...          NaN          NaN
9217526     1000505  2017-01-02 13:36:09     ...          NaN          NaN
9217527     1000505  2017-01-02 13:36:10     ...          NaN          NaN

[5 rows x 14 columns]

In [34]:
def count_hp_util(v):
    if v>=0.5 and v<0.6:
        return 1
    elif v>=0.6 and v<0.7:
        return 2
    elif v>=0.7 and v<0.8:
        return 3
    elif v>=0.8 and v<0.9:
        return 4
    else:
        return 0
dfdrive['act_hp_util_bins']=dfdrive['act_hp_util'].apply(count_hp_util)

In [35]:
dfdrive['rpm_util']=dfdrive['rpm']/dfdrive['max_horsepower_rpm']

In [36]:
def count_hp_util(v):
    if v>=0.5 and v<0.6:
        return 1
    elif v>=0.6 and v<0.7:
        return 2
    else:
        return 0
dfdrive['rpm_util_bin']=dfdrive['rpm_util'].apply(count_hp_util)

In [37]:
def count_torque_bins1(series):
    return len([x for x in series if x==1])
def count_torque_bins2(series):
    return len([x for x in series if x==2])
def count_torque_bins3(series):
    return len([x for x in series if x==3])
def count_torque_bins4(series):
    return len([x for x in series if x==4])

def count_hp_util1(series):
    return len([x for x in series if x==1])
def count_hp_util2(series):
    return len([x for x in series if x==2])
def count_hp_util3(series):
    return len([x for x in series if x==3])
def count_hp_util4(series):
    return len([x for x in series if x==4])

def rpm_util1(series):
    return len([x for x in series if x==1])
def rpm_util2(series):
    return len([x for x in series if x==2])

xx=dfdrive.groupby(by='vehicle_id').resample('W-Mon', on='week_start_date')

engi_final=xx.agg(count_torque_bins1)['torque_bin'].reset_index().rename(columns={'torque_bin':'ft_torque_util_60pct_s'})
engi_final=engi_final.merge(xx['torque_bin'].agg(count_torque_bins2).reset_index().rename(columns={'torque_bin':'ft_torque_util_70pct_s'}),on=['vehicle_id','week_start_date'],how='left')
engi_final=engi_final.merge(xx['torque_bin'].agg(count_torque_bins3).reset_index().rename(columns={'torque_bin':'ft_torque_util_80pct_s'}),on=['vehicle_id','week_start_date'],how='left')
engi_final=engi_final.merge(xx['torque_bin'].agg(count_torque_bins4).reset_index().rename(columns={'torque_bin':'ft_torque_util_90pct_s'}),on=['vehicle_id','week_start_date'],how='left')

In [38]:
engi_final.head()

vehicle_id          ...           ft_torque_util_90pct_s
0     1000500          ...                              7.0
1     1000500          ...                             39.0
2     1000500          ...                             24.0
3     1000500          ...                             33.0
4     1000500          ...                            112.0

[5 rows x 6 columns]

In [39]:
engi_final=engi_final.merge(xx['act_hp_util_bins'].agg(count_hp_util1).reset_index().rename(columns={'act_hp_util_bins':'ft_horsepower_util_50pct_s'}),on=['vehicle_id','week_start_date'],how='left')
engi_final=engi_final.merge(xx['act_hp_util_bins'].agg(count_hp_util2).reset_index().rename(columns={'act_hp_util_bins':'ft_horsepower_util_60pct_s'}),on=['vehicle_id','week_start_date'],how='left')
engi_final=engi_final.merge(xx['act_hp_util_bins'].agg(count_hp_util3).reset_index().rename(columns={'act_hp_util_bins':'ft_horsepower_util_70pct_s'}),on=['vehicle_id','week_start_date'],how='left')
engi_final=engi_final.merge(xx['act_hp_util_bins'].agg(count_hp_util4).reset_index().rename(columns={'act_hp_util_bins':'ft_horsepower_util_80pct_s'}),on=['vehicle_id','week_start_date'],how='left')

engi_final=engi_final.merge(xx['rpm_util_bin'].agg(rpm_util1).reset_index().rename(columns={'rpm_util_bin':'ft_rpm_util_50pct_s'}),on=['vehicle_id','week_start_date'],how='left')
engi_final=engi_final.merge(xx['rpm_util_bin'].agg(rpm_util1).reset_index().rename(columns={'rpm_util_bin':'ft_rpm_util_60pct_s'}),on=['vehicle_id','week_start_date'],how='left')
engi_final.head()

vehicle_id         ...         ft_rpm_util_60pct_s
0     1000500         ...                         0.0
1     1000500         ...                         0.0
2     1000500         ...                         0.0
3     1000500         ...                         0.0
4     1000500         ...                         0.0

[5 rows x 12 columns]

In [40]:
vehicle.shape

(7, 17)

In [43]:
engi_final['week_start_date']=engi_final['week_start_date'].astype('category')
engi_final['vehicle_id']=engi_final['vehicle_id'].astype('category')
engine['vehicle_id']=engine['vehicle_id'].astype('category')
engine['week_start_date']=engine['week_start_date'].astype('category')

In [44]:
# final_eng=final[['vehicle_id','week_start_date']].merge(engine,on=['vehicle_id','week_start_date'],how='left')
final_eng=engi_final.merge(engine.drop(['ft_torque_util_60pct_s',
       'ft_torque_util_70pct_s', 'ft_torque_util_80pct_s',
       'ft_torque_util_90pct_s', 'ft_horsepower_util_50pct_s',
       'ft_horsepower_util_60pct_s', 'ft_horsepower_util_70pct_s',
       'ft_horsepower_util_80pct_s', 'ft_rpm_util_50pct_s',
       'ft_rpm_util_60pct_s'],axis=1),on=['vehicle_id','week_start_date'],how='left').sort_values(by=['vehicle_id','week_start_date'])

In [45]:
# engine['week_start_date']=final['date']
engine.head()

vehicle_id         ...         ft_rpm_util_60pct_s
0    1000501         ...                         434
1    1000503         ...                        7534
2    1000504         ...                        7534

[3 rows x 12 columns]

In [46]:
final_eng.head()

vehicle_id         ...         ft_rpm_util_60pct_s
0     1000500         ...                         0.0
1     1000500         ...                         0.0
2     1000500         ...                         0.0
3     1000500         ...                         0.0
4     1000500         ...                         0.0

[5 rows x 12 columns]

In [47]:
final_eng.fillna(0,inplace=True)
final_eng.head()

vehicle_id         ...         ft_rpm_util_60pct_s
0     1000500         ...                         0.0
1     1000500         ...                         0.0
2     1000500         ...                         0.0
3     1000500         ...                         0.0
4     1000500         ...                         0.0

[5 rows x 12 columns]

In [48]:
final_eng['week_start_date']=final_eng['week_start_date'].apply(lambda x: x - pd.DateOffset(days=7))
final_eng.head()

vehicle_id         ...         ft_rpm_util_60pct_s
0     1000500         ...                         0.0
1     1000500         ...                         0.0
2     1000500         ...                         0.0
3     1000500         ...                         0.0
4     1000500         ...                         0.0

[5 rows x 12 columns]

In [49]:
final_eng.to_csv('engine_features.csv',index=False)

# Weather Features

### Basic date done - no features

In [50]:
final=dfdrive.groupby(by='vehicle_id').resample('W-Mon', on='week_start_date').sum().rename(columns={'week_start_date':'week_start_date1',
                                                                                               'vehicle_id':'vehicle_id1'}).reset_index()[['vehicle_id','week_start_date']]
final['week_start_date']=pd.to_datetime(final['week_start_date'])
final['week_start_date']=final['week_start_date'].apply(lambda x: x - pd.DateOffset(days=7))


In [51]:
final.head()

vehicle_id week_start_date
0     1000500      2016-12-26
1     1000500      2017-01-02
2     1000500      2017-01-09
3     1000500      2017-01-16
4     1000500      2017-01-23

In [52]:
final['week_start_date']=final['week_start_date'].dt.strftime('%m/%d/%y')

In [53]:
# final['week_start_date']

In [54]:
final['week_start_date']=final['week_start_date'].astype('category')
final['vehicle_id']=final['vehicle_id'].astype('category')

weather['week_start_date']=weather['week_start_date'].astype('category')
weather['vehicle_id']=weather['vehicle_id'].astype('category')


final[['vehicle_id','week_start_date']].merge(weather,on=['vehicle_id','week_start_date'],how='left').fillna(0).to_csv('weather_features.csv',index=False)

In [55]:
final[['vehicle_id','week_start_date']].merge(weather,on=['vehicle_id','week_start_date'],how='left').fillna(0)

vehicle_id             ...             total_heavy_rain_driving_km
0       1000500             ...                                     0.0
1       1000500             ...                                     0.0
2       1000500             ...                                     0.0
3       1000500             ...                                     0.0
4       1000500             ...                                     0.0
5       1000500             ...                                     0.0
6       1000500             ...                                     0.0
7       1000500             ...                                     0.0
8       1000500             ...                                     0.0
9       1000500             ...                                     0.0
10      1000501             ...                                     0.0
11      1000501             ...                                     0.0
12      1000501             ...                                     0.0
13      1000501             ...                                     0.0
14      1000501             ...                                     0.0
15      1000501             ...                                     0.0
16      1000501             ...                                     0.0
17      1000501             ...                                     0.0
18      1000501             ...                                     0.0
19      1000502             ...                                     0.0
20      1000502             ...                                     0.0
21      1000502             ...                                     0.0
22      1000502             ...                                     0.0
23      1000502             ...                                     0.0
24      1000502             ...                                     0.0
25      1000502             ...                                     0.0
26      1000502             ...                                     0.0
27      1000502             ...                                     0.0
28      1000502             ...                                     0.0
29      1000503             ...                                     0.0
..          ...             ...                                     ...
156     1000516             ...                                     0.0
157     1000516             ...                                     0.0
158     1000517             ...                                     0.0
159     1000517             ...                                     0.0
160     1000517             ...                                     0.0
161     1000517             ...                                     0.0
162     1000517             ...                                     0.0
163     1000517             ...                                     0.0
164     1000517             ...                                     0.0
165     1000517             ...                                     0.0
166     1000517             ...                                     0.0
167     1000517             ...                                     0.0
168     1000518             ...                                     0.0
169     1000518             ...                                     0.0
170     1000518             ...                                     0.0
171     1000518             ...                                     0.0
172     1000518             ...                                     0.0
173     1000518             ...                                     0.0
174     1000518             ...                                     0.0
175     1000518             ...                                     0.0
176     1000518             ...                                     0.0
177     1000518             ...                                     0.0
178     1000519             ...                                     0.0
179     1000519             ...              